In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbml registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
import numpy as np 

y = np.array([0.5, 1.0, 0.75, 0.60, 0.66])

average_y = np.mean(y)

# Calculate the MSE for each value in y with respect to the average
mse_scores = (y - average_y) ** 2

mse_scores, average_y

(array([0.040804, 0.088804, 0.002304, 0.010404, 0.001764]), 0.7020000000000001)

In [3]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

"TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:\n\n{('emitter',): [{\n    'copasi': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 0.0,\n                'plasmin': 0.0,\n                'single intact chain urokinase-type plasminogen activator': 0.0,\n                'two-chain urokinase-type plasminogen activator': 0.0,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0\n        },\n        ...\n    ],\n    'amici': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 1.1758171177387002e+16,\n                'plasmin': 1096150505274.1506,\n                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,\n                'two-chain urokinase-type plasminogen activator': 80249.33829510311,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0},\n        },\n 

In [4]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
model_fp = f'../biosimulator_processes/model_files/sbml/{biomodel_id}_url.xml'
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'amici']

In [5]:
biomodel_id = 'BIOMD0000000744'
document = {
    'sbml': {
        '_type': 'step',
        'address': 'local:get_sbml',
        'config': {
            'biomodel_id': biomodel_id
        },
        'inputs': {
                'biomodel_id': ['biomodel_id_store']
        },
        'outputs': {
                'sbml_model_fp': ['sbml_model_fp_store'],
        }
    },
    'ode_compare': {
        '_type': 'step',
        'address': 'local:compare_ode_step',
        'inputs': {
            'data': ['sbml_model_fp_store']
        },
        'outputs': {
            'comparison': ['comparison_store']
        }
    },
    'copasi': {
        '_type': 'process',
        'address': 'local:_process',
        'inputs': {
            'source': ['comparison_store']
        },
        'outputs': {
            'names': ['names_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'comparison': 'string',
                'names': 'list[string]'
            },
        },
        'inputs': {
            'comparison': ['comparison_store'],
            'names': ['names_store']
        }
    }
}

In [6]:
wf = Composite(config={'state': document}, core=CORE)

In [7]:
wf.run(10)
state = wf.gather_results()

There is no model, creating a new one


BIOMD0000000744
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml
True
the source: /var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmc

In [8]:
state

{('emitter',): [{'comparison': '/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml',
   'names': []}]}

In [9]:
state

{('emitter',): [{'comparison': '/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpmcmt5lcr/BIOMD0000000744_url.xml',
   'names': []}]}